# Multiclass Logistic Regression Classifier

In [179]:
# Libraries
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

In [180]:
# Get directories & labels
train_dir = "C:/Users/bodin/School/MATH 5680/Projec/tumor_images/Training"
test_dir = "C:/Users/bodin/School/MATH 5680/Projec/tumor_images/Testing"

labels = ["glioma_tumor","meningioma_tumor","no_tumor","pituitary_tumor"]

In [181]:
# Create list of data
train_img_list = []
test_img_list = []

test_label_list = []
train_label_list = []

# Training dataset
for label in labels:
    for img_file in os.listdir(train_dir + '/' + label):
        img = cv2.imread(train_dir + '/' + label + '/' + img_file)
        img = cv2.resize(img, (200,200))
        train_img_list.append(img)
        train_label_list.append(label)

# Testing dataset
for label in labels:
    for img_file in os.listdir(test_dir + '/' + label):
        img = cv2.imread(test_dir + '/' + label + '/' + img_file)
        img = cv2.resize(img, (200,200))
        test_img_list.append(img)
        test_label_list.append(label)

In [182]:
# Encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

train_label_list = label_encoder.fit_transform(train_label_list)
test_label_list = label_encoder.fit_transform(test_label_list)

# Convert images to int64 array
train_img_list = np.array(train_img_list).astype(np.int64)
test_img_list = np.array(test_img_list).astype(np.int64)

In [183]:
# Value Counts for each type of tumor
## Pituitary = 3
## Glioma = 0
## Meningioma = 1
## None = 2
pd.Series(train_label_list).value_counts()

3    827
0    826
1    822
2    395
dtype: int64

In [184]:
# Convert images from (200, 200, 3) to column vector (120000)
X_train = train_img_list.reshape(len(train_img_list), -1)
X_test = test_img_list.reshape(len(test_img_list), -1)
Y_train = train_label_list
Y_test = test_label_list

In [185]:
# Standardize values between 0-1
X_train = X_train / 255
X_test = X_test / 255

In [186]:
### Not necessary in current state, may delete ###

#from sklearn.decomposition import PCA
#pca = PCA(.98)

#pca_train = pca.fit_transform(X_train)
#pca_test = pca.transform(X_test)

#pca_train = X_train
#pca_test = X_test

In [187]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1, max_iter=500)
lr.fit(pca_train, Y_train);

C:\Users\bodin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [188]:
print(f"Training score: {lr.score(pca_train, Y_train)}")
print(f"Testing score: {lr.score(pca_test, Y_test)}")

Training score: 1.0
Testing score: 0.7360406091370558
